La **Regresión Logística** se encuentra implementada en dos liberías. La primera es la que venimos utilizando hasta el día de hoy. Es Scikit-Learn. No obstante, en dicha librería no existe implementada la inferencia estadística. Es decir, si queremos calcular el p-value de cada una de las variables para saber si alguna variable es o no relevante en el modelo. Esto no se encuentra desarrollado. Por lo tanto se deberá crear una función específica para poder calcularlo.

Otra librería en donde se encuentra implementada es la statsmodel.

In [3]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings("ignore")

import os
os.chdir('../')

from sklearn.model_selection import train_test_split

from scipy.stats import norm
from sklearn.linear_model import LogisticRegression


In [4]:
train = pd.read_csv("./data/titanic/02 - preprocesada/train.csv")
X = train.drop(columns = ["Survived", "PassengerId"])
y = train.Survived
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [5]:
y_train.values

array([0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0.,
       1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0.,
       0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0.,
       1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 1., 1.,
       0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 1.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1.,
       0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
       1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0.,
       1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
       1., 1., 0., 1., 0.

In [6]:
def logit_pvalue(model, x):
    """ Calculate z-scores for scikit-learn LogisticRegression.
    parameters:
        model: fitted sklearn.linear_model.LogisticRegression with intercept and large C
        x:     matrix on which the model was fit
    This function uses asymtptics for maximum likelihood estimates.
    """
    p = model.predict_proba(x)
    n = len(p)
    m = len(model.coef_[0]) + 1
    coefs = np.concatenate([model.intercept_, model.coef_[0]])
    x_full = np.matrix(np.insert(np.array(x), 0, 1, axis = 1))
    ans = np.zeros((m, m))
    for i in range(n):
        ans = ans + np.dot(np.transpose(x_full[i, :]), x_full[i, :]) * p[i,1] * p[i, 0]
    vcov = np.linalg.inv(np.matrix(ans))
    se = np.sqrt(np.diag(vcov))
    t =  coefs/se  
    p = (1 - norm.cdf(abs(t))) * 2
    return p

In [9]:
model

LogisticRegression()

In [8]:
#X_train = X_train.drop(columns=["titulo_Royalty"])
model = LogisticRegression().fit(X_train, y_train)
print(logit_pvalue(model, X_train))

[0.99999992 0.69887102 0.00151742 0.06028421 0.11725727        nan
        nan        nan 0.99999977        nan 0.99999983 1.
 0.99999987 0.99999993 0.99999991 0.99999999 0.99999998 0.99999999
 1.         1.         0.99999999 1.         0.99999998 1.
 0.99999998 1.         0.03097363]


In [121]:
import statsmodels.api as sm
logit_model=sm.Logit(y_test,X_test.astype(float))
result=logit_model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.347619
         Iterations 22
                           Logit Regression Results                           
Dep. Variable:               Survived   No. Observations:                  295
Model:                          Logit   Df Residuals:                      272
Method:                           MLE   Df Model:                           22
Date:                Sat, 29 Aug 2020   Pseudo R-squ.:                  0.4855
Time:                        23:53:49   Log-Likelihood:                -102.55
converged:                       True   LL-Null:                       -199.32
Covariance Type:            nonrobust   LLR p-value:                 2.072e-29
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Age                     -0.1494      0.068     -2.210      0.027      -0.282      -0.01

In [131]:
X_test.groupby("Sex_male").size()

Sex_male
0    108
1    187
dtype: int64

In [118]:
X_train.titulo_Master.std()

0.20814040848186624

In [120]:
p[0,1]

0.2937577215096096